<a href="https://colab.research.google.com/github/MuhammadAbuSufian/colab/blob/main/AGIQA_3K.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import necessary libraries
!pip install image-reward
#!pip install torch==1.10.0+cu102 torchvision==0.11.1+cu102 torchaudio==0.10.0+cu102 -f https://download.pytorch.org/whl/cu102/torch_stable.html
!pip install git+https://github.com/openai/CLIP.git

import pandas as pd
from transformers import AutoProcessor, AutoModel
from PIL import Image
import torch
import requests
from io import BytesIO
import os
import ImageReward as RM

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Load the CSV file
raw_data = pd.read_csv('/content/drive/MyDrive/Colab/Data/data.csv')
data = raw_data

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Initialize an empty list to store results
results = []
expected_results = []

model = RM.load("ImageReward-v1.0")

# Iterate over each row in the dataframe
for index, row in data.iterrows():
    image_name = row['name']
    prompt = row['prompt']
    human_feedback_score = row['std_align']


    # Construct the path to the image file
    image_path = os.path.join('/content/drive/MyDrive/Colab/Images/AGIQA-3K', image_name)

    # Load the image
    try:
        with Image.open(image_path) as img:
            rewards = model.score(prompt, img)
            normalized_reward = 1 / (1 + np.exp(-rewards))  # Apply sigmoid function
            results.append(normalized_reward)
            expected_results.append(human_feedback_score)
    except IOError:
        print(f"Failed to load image {image_name}")

# Convert results to a DataFrame for easy viewing or further processing

print(results)
print(expected_results)

In [ ]:
#Calculate PLCC
import numpy as np
from scipy.stats import pearsonr, spearmanr, kendalltau

subjective_scores = np.array(expected_results, dtype=float)
objective_scores = np.array(results, dtype=float)

print(subjective_scores.size)
print(objective_scores.size)

# Calculate PLCC
plcc, _ = pearsonr(subjective_scores, objective_scores)

# Calculate SROCC
srocc, _ = spearmanr(subjective_scores, objective_scores)

# Calculate KROCC
krocc, _ = kendalltau(subjective_scores, objective_scores)

print("PLCC:", plcc)
print("SROCC:", srocc)
print("KROCC:", krocc)